In [1]:
import faiss
import numpy as np

In [2]:
np.random.seed(97)

---

# `IndexFlatL2` 

 `IndexFlatL2` - простой индекс, который ищет ближайшего соседа полным перебором.

# Как для одного вектора найти его ближайшего соседа в "базе данных" (индексе)?

In [3]:
dim = 512  
num_vectors = 10000 
k = 1

vectors = np.random.random((num_vectors, dim)).astype('float32')
index = faiss.IndexFlatL2(dim)
index.add(vectors)

query_vector = np.random.random(dim).astype('float32')
query = np.array([query_vector])

distances, indices = index.search(query, k)

nearest_vector = vectors[indices[0][0]]

---

In [4]:
dim = 512            # размерность векторов
num_vectors = 10000  # количество векторов в индексе
k = 1                # количество искомых соседей

In [5]:
# Создадим "базу данных" (индекс) из случайных векторов:

vectors = np.random.random((num_vectors, dim)).astype('float32')
index = faiss.IndexFlatL2(dim)
index.add(vectors)

print(f'- vectors.shape: {vectors.shape}')

- vectors.shape: (10000, 512)


In [6]:
# Создадим случайный вектор, для которого будем искать ближайшего соседа:

query_vector = np.random.random(dim).astype('float32')

print(f'- query_vector.shape: {query_vector.shape}')

- query_vector.shape: (512,)


In [7]:
# Создадим запрос в "базу данных":

query = np.array([query_vector])

print(f'- query.shape: {query.shape}')

- query.shape: (1, 512)


In [8]:
# Найдем индекс ближайшего соседа в нашей "базе данных" (индексе) и расстояние от него до искомого вектора:

distances, indices = index.search(query, k)

print(f'- indexes: {indices}')
print(f'- distances: {distances}')

- indexes: [[8273]]
- distances: [[70.1127]]


In [9]:
# Теперь, зная индекс ближайшего соседа мы можем сохранить ближайший вектор в отдельную переменную:

nearest_vector = vectors[indices[0][0]]

print(f'- nearest_vector.shape: {nearest_vector.shape}')

- nearest_vector.shape: (512,)


---

---

# Как для нескольких векторов найти их ближайший соседей в "базе данных" (индексе)?

In [10]:
dim = 512  
num_vectors = 10000
num_query = 5
k = 1

vectors = np.random.random((num_vectors, dim)).astype('float32')
index = faiss.IndexFlatL2(dim)
index.add(vectors)

query = np.random.random([num_query,dim]).astype('float32')

distances, indices = index.search(query, k)

---

In [11]:
dim = 512            # рассмотрим произвольные векторы размерности 512
num_vectors = 10000  # количество векторов в индексе
num_query = 5        # количество векторов в выборке для поиска
k = 1                # количество искомых соседей

In [12]:
# Создадим "базу данных" (индекс) из случайных векторов:

vectors = np.random.random((num_vectors, dim)).astype('float32')
index = faiss.IndexFlatL2(dim)
index.add(vectors)

print(f'- vectors.shape: {vectors.shape}')

- vectors.shape: (10000, 512)


In [13]:
# Создадим запрос в "базу данных":

query = np.random.random([num_query, dim]).astype('float32')
print(f'- query.shape: {query.shape}')

- query.shape: (5, 512)


In [14]:
# Найдем индексы ближайших соседей в нашей "базе данных" (индексе) и расстояние от них до искомых векторов:

distances, indices = index.search(query, k)

print('indexes: \t distances:')
for  i, d in zip (indices, distances):
    print(f'{i[0]} \t \t {(d[0])}')

indexes: 	 distances:
5240 	 	 70.99066925048828
6815 	 	 71.5201416015625
8791 	 	 67.58242797851562
4948 	 	 68.54409790039062
2386 	 	 68.1894760131836


---

### Проверим что для векторов из "базы данных" (индекса) ближашими будут они сами.

In [15]:
# Возьмем случайные индексы элементов из "бызы данных" (индекса):

query_indexes = np.random.choice(num_vectors, num_query)
                                
print(f'query_indexes: {query_indexes}')

query_indexes: [ 728 4702 7027 2194 9222]


In [16]:
# Возьмем вектора соответствующие выбранным индексам:

query = vectors[query_indexes]

In [17]:
# Найдем ближайших соседей для векторов которые уже есть в "базе данных" (индексе):

distances, indices = index.search(query, k)

print('query_indexes: \t indexes: \t distances:')
for  qi, i, d in zip (query_indexes, indices, distances):
    print(f'{qi} \t \t {i[0]} \t \t {d[0]}')

query_indexes: 	 indexes: 	 distances:
728 	 	 728 	 	 0.0
4702 	 	 4702 	 	 0.0
7027 	 	 7027 	 	 0.0
2194 	 	 2194 	 	 0.0
9222 	 	 9222 	 	 0.0


---

---

# Как построить индекс если вектора не помещаются в память?

In [18]:
# Обычный способ, мы будем его использовать для проверки результата:

dim = 512
num_vectors = 10000
num_query = 3
k = 10

vectors = np.random.random((num_vectors, dim)).astype(np.float32)
index = faiss.IndexFlatL2(dim)
index.add(vectors)

query = np.random.random((num_query, dim)).astype(np.float32)

dists, ids = index.search(query, k)

In [19]:
# Разделение "базы данных" (индекса) на несколько частей,
# поиск ближайших соседей в каждой части и объединение результатов:

index_1 = faiss.IndexFlatL2(dim)
index_1.add(vectors[:5000])   

index_2 = faiss.IndexFlatL2(dim)
index_2.add(vectors[5000:])   

dists_1, ids_1 = index_1.search(query, k)
dists_2, ids_2 = index_2.search(query, k)

result_heap = faiss.ResultHeap(num_query, k)

result_heap.add_result(D=dists_1, I=ids_1)
result_heap.add_result(D=dists_2, I=ids_2 + 5000)  

result_heap.finalize()

new_dists = result_heap.D
new_ids = result_heap.I
assert np.array_equal(dists, new_dists)
assert np.array_equal(ids, new_ids)

---

In [20]:
# Строим индекс для первой половины данных:

index_1 = faiss.IndexFlatL2(dim)
index_1.add(vectors[:5000])   

# Строим индекс для второй половины данных:

index_2 = faiss.IndexFlatL2(dim)
index_2.add(vectors[5000:])  

In [21]:
# Ищем ближайших соседей в каждой половине:

dists_1, ids_1 = index_1.search(query, k)
dists_2, ids_2 = index_2.search(query, k)

In [22]:
# Объединяем результаты:

result_heap = faiss.ResultHeap(num_query, k)

result_heap.add_result(D=dists_1, I=ids_1)
result_heap.add_result(D=dists_2, I=ids_2 + 5000)  
result_heap.finalize()

# Сохраняем результаты в отдельные переменные:

dists = result_heap.D
ids = result_heap.I